# Google form analysis

Analysis of results extracted from Google forms in csv format.

## Table of Contents


[Preparation](#preparation)

[Constants](#constants)
   
   - [en-fr gform merge](#merge)

[Functions](#functions)
   
   - [general purpose](#genpurpose)
   
   - [score](#score)
   
   - [checkpoint validation](#checkvalidation)
   
   - [p(answered question N | answered question P)](#condproba)

## Preparation
<a id=preparation />

In [1]:
%run "../Functions/2. Game sessions.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,

# Constants
<a id=constants />

In [2]:
gformPath = "../../data/Google forms/"
genericFormName = '2017-11-15-gform-'
enSuffix = 'en'
frSuffix = 'fr'
csvSuffix = '.csv'
gformEN = pd.read_csv(gformPath + genericFormName + enSuffix + csvSuffix)
gformFR = pd.read_csv(gformPath + genericFormName + frSuffix + csvSuffix)

In [3]:
# special user ids
userIDThatDidNotAnswer = '"001c95c6-8207-43dc-a51b-adf0c6e005d7"'

userID1AnswerEN = '"00dbbdca-d86c-4bc9-803c-0602e0153f68"'
userIDAnswersEN = '"5977184a-1be2-4725-9b48-f2782dc03efb"'
userID1ScoreEN = '"6b5d392d-b737-49ef-99af-e8c445ff6379"'
userIDScoresEN = '"5ecf601d-4eac-433e-8056-3a5b9eda0555"'

userID1AnswerFR = '"2734a37d-4ba5-454f-bf85-1f7b767138f6"'
userIDAnswersFR = '"01e85778-2903-447b-bbab-dd750564ee2d"'
userID1ScoreFR = '"3d733347-0313-441a-b77c-3e4046042a53"'
userIDScoresFR = '"58d22690-8604-41cf-a5b7-d71fb3b9ad5b"'

userIDAnswersENFR = '"a7936587-8b71-43b6-9c61-17b2c2b55de3"'

### en-fr gform merge
<a id=merge />

In [4]:
gformEN['Language'] = pd.Series('en', index=gformEN.index)
gformFR['Language'] = pd.Series('fr', index=gformFR.index)
gformFR.columns = gformEN.columns
gform = pd.concat([gformEN, gformFR])
gform.index = range(0, len(gform.index))

In [5]:
gform.head(2)

,Timestamp,Are you interested in video games?,Do you play video games?,How old are you?,What is your gender?,How long have you studied biology?,Are you interested in biology?,"Before playing Hero.Coli, had you ever heard about synthetic biology?","Before playing Hero.Coli, had you ever heard about BioBricks?",Have you ever played an older version of Hero.Coli before?,...,What does this device do?.6,What does this device do?.7,"Guess: what would a device producing l-arabinose do, if it started with a l-arabinose-induced promoter?",Guess: the bacterium would glow yellow...,What is the species of the bacterium of the game?,What is the scientific name of the tails of the bacterium?,Find the antibiotic:,You can write down remarks here.,Do not edit - pre-filled anonymous ID,Language
0,2017/08/18 3:22:26 PM GMT+1,Extremely,Extremely,23,Female,Until bachelor's degree,Extremely,Yes,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,Your game is amazing. One suggestion. In sever...,8d352896-a3f1-471c-8439-0f426df901c1,en
1,2017/09/06 1:28:03 PM GMT+1,Moderately,Moderately,28,Other,Until the end of high school,Moderately,Yes,NaN,No,...,It generates antibiotic resistance,It generates green fluorescence in presence of...,"After being induced, it would produce more and...",I don't know,E. Coli,Flagella,Ampicillin,"This was a nice educational game, thank you fo...",7037c5b2-c286-498e-9784-9a061c778609,en


In [6]:
gform.tail(2)

,Timestamp,Are you interested in video games?,Do you play video games?,How old are you?,What is your gender?,How long have you studied biology?,Are you interested in biology?,"Before playing Hero.Coli, had you ever heard about synthetic biology?","Before playing Hero.Coli, had you ever heard about BioBricks?",Have you ever played an older version of Hero.Coli before?,...,What does this device do?.6,What does this device do?.7,"Guess: what would a device producing l-arabinose do, if it started with a l-arabinose-induced promoter?",Guess: the bacterium would glow yellow...,What is the species of the bacterium of the game?,What is the scientific name of the tails of the bacterium?,Find the antibiotic:,You can write down remarks here.,Do not edit - pre-filled anonymous ID,Language
71,2017/10/28 2:32:13 PM GMT+1,Moyennement,Moyennement,11,Homme,"Jamais, pas même au collège",Un peu,Non,Non,Non,...,Elle permet de résister aux antibiotiques,Elle produit la fluorescence verte en présence...,Je ne sais pas,Je ne sais pas,E. Coli,Des flagelles,Je ne sais pas,NaN,bdc1c38d-dce7-4c00-8ea2-9514643bc09f,fr
72,2017/11/14 8:10:15 AM GMT+1,Beaucoup,Moyennement,44,Homme,Jusqu'au brevet,Beaucoup,Oui,Non,Non,...,Elle permet de résister aux antibiotiques,Elle produit la fluorescence verte en présence...,Je ne sais pas,Je ne sais pas,E. Coli,Des flagelles,Ampicilline,NaN,deb089c0-9be3-4b75-9b27-28963c77b10c,fr


In [7]:
#localplayerguidkey = 'Ne pas modifier - identifiant anonyme prérempli'
localplayerguidkey = 'Do not edit -  pre-filled anonymous ID'
localplayerguidindex = gform.columns.get_loc(localplayerguidkey)
localplayerguidindex

41

In [8]:
firstEvaluationQuestionKey = 'In order to modify the abilities of the bacterium, you have to...'
firstEvaluationQuestionIndex = gform.columns.get_loc(firstEvaluationQuestionKey)
firstEvaluationQuestionIndex

13

In [9]:
answersColumnNameStem = "answers"
correctionsColumnNameStem = "corrections"

# Functions
<a id=functions />

## general purpose
<a id=genpurpose />

In [17]:
#user id in format localplayerguid = '"8d352896-a3f1-471c-8439-0f426df901c1"'

def getRedMetricsGUIDFormat( gformGUID ):
    return gformGUID.replace('"','')

def getGFormGUIDFormat( redmetricsGUID ):
    return '"' + redmetricsGUID + '"'

def getAllResponders():
    userIds = gform[localplayerguidkey].values
    result = []
    for id in userIds:
        result.append('"' + id + '"')
    return result

def hasAnswered( userId ):
    return userId.replace('"','') in gform[localplayerguidkey].values

def getAnswers( userId ):
    answers = gform[gform[localplayerguidkey]==userId.replace('"','')]
    _columnAnswers = answers.T
    
    if 0 != len(answers):
        _newColumns = []
        for column in _columnAnswers.columns:
            _newColumns.append(answersColumnNameStem + str(column))
        _columnAnswers.columns = _newColumns
    else:
        # user has never answered
        print("user " + str(userId) + " has never answered")
        
    return _columnAnswers

## sessions

In [21]:
# when did the user answer the questionnaire? After playing, before playing, undefined?
answerTemporalities = ['before', 'after', 'undefined'];
def setAnswerTemporalities( _df ):
    _df['Temporality'] = _df['Timestamp']
    for _index in _df.index:
        _firstEventDate = getFirstEventDate(getGFormGUIDFormat(_df[localplayerguidkey][_index]))
        _df['Temporality'][_index] = getTemporality(_df['Temporality'][_index],_firstEventDate)

# when did the user answer the questionnaire? 
# After gameEventDate, before gameEventDate, undefined?
# answerDate is assumed to be a string coding for tz='Europe/Berlin' time
# gameEventDate is assumed to be UTC
def getTemporality( answerDate, gameEventDate ):    
    if(gameEventDate == pd.Timestamp.max.tz_localize('utc')):
        return answerTemporalities[2]
    
    tzAnswerDate = pd.to_datetime(answerDate).tz_localize('Europe/Berlin')        
    if(tzAnswerDate < gameEventDate):
        return answerTemporalities[0]
    elif (tzAnswerDate > gameEventDate):
        return answerTemporalities[1]
    else:
        return answerTemporalities[2]

## score
<a id=score />

In [12]:
def getCorrections( userId ):
    _columnAnswers = getAnswers( userId )
    
    if 0 != len(_columnAnswers.columns):

        _questionsCount = len(_columnAnswers.values)

        for _columnName in _columnAnswers.columns:
            if answersColumnNameStem in _columnName:
                _answerNumber = _columnName.replace(answersColumnNameStem,"")
                _newCorrectionsColumnName = correctionsColumnNameStem + _answerNumber
                _columnAnswers[_newCorrectionsColumnName] = _columnAnswers[_columnName]

                for _index in range(0, firstEvaluationQuestionIndex):
                    _columnAnswers[_newCorrectionsColumnName][_index] = 'NaN'
                for _index in range(firstEvaluationQuestionIndex, len(_columnAnswers.values)-3):
                    _columnAnswers[_newCorrectionsColumnName][_index] =  \
                        str(_columnAnswers[_columnName][_index]).startswith(str(correctAnswers[_columnAnswers[_columnName]['Language']][_index]))
                        
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-3] = 'NaN'
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-2] = 'NaN'
                _columnAnswers[_newCorrectionsColumnName][_questionsCount-1] = 'NaN'
        return _columnAnswers
    else:
        # user has never answered
        print("can't give correct answers")
    return _columnAnswers

#function that returns the score from user id
def getScore( userId ):
    _score = []
    
    if hasAnswered( userId ):
        _columnAnswers = getCorrections(userId)
        for _columnName in _columnAnswers.columns:
            # only work on corrected columns
            if correctionsColumnNameStem in _columnName:
                _score.append((_columnAnswers[_columnName]).value_counts()[True])
    else:
        print("user " + str(userId) + " has never answered")
    return _score

## checkpoint validation
<a id=checkvalidation />

In [13]:
#function that returns the list of checkpoints from user id
def getValidatedCheckpoints( userId ):
    _validatedCheckpoints = []
    
    if hasAnswered( userId ):
        _columnAnswers = getCorrections( userId)
        
        for _columnName in _columnAnswers.columns:
            # only work on corrected columns
            if correctionsColumnNameStem in _columnName:        
                _questionnaireValidatedCheckpointsPerQuestion = pd.Series(np.nan, index=range(len(checkpointQuestionMatching)))

                for _index in range(0, len(_questionnaireValidatedCheckpointsPerQuestion)):
                    if _columnAnswers[_columnName][_index]==True:
                        _questionnaireValidatedCheckpointsPerQuestion[_index] = checkpointQuestionMatching['checkpoint'][_index]
                    else:
                        _questionnaireValidatedCheckpointsPerQuestion[_index] = ''

                _questionnaireValidatedCheckpoints = _questionnaireValidatedCheckpointsPerQuestion.unique()
                _questionnaireValidatedCheckpoints = _questionnaireValidatedCheckpoints[_questionnaireValidatedCheckpoints!='']
                _questionnaireValidatedCheckpoints = pd.Series(_questionnaireValidatedCheckpoints)
                _questionnaireValidatedCheckpoints = _questionnaireValidatedCheckpoints.sort_values()
                _questionnaireValidatedCheckpoints.index = range(0, len(_questionnaireValidatedCheckpoints))
                
                _validatedCheckpoints.append(_questionnaireValidatedCheckpoints) 
    else:
        print("user " + str(userId) + " has never answered")
    return pd.Series(_validatedCheckpoints)

def getValidatedCheckpointsCounts( userId ):
    _validatedCheckpoints = getValidatedCheckpoints(userId)
    _counts = []
    for checkpointsList in _validatedCheckpoints:
        _counts.append(len(checkpointsList))
    return _counts

def getNonValidated( checkpoints ):
    _validationLists = []
    
    if 0!=len(checkpoints):
        for _validation in checkpoints:
            _result = pd.Series(np.setdiff1d(validableCheckpoints.values, _validation.values))
            _result = _result[_result != '']
            _result.index = range(0, len(_result))
            _validationLists.append(_result)
        return pd.Series(_validationLists)
    else:
        return validableCheckpoints

def getNonValidatedCheckpoints( userId ):
    validated = getValidatedCheckpoints( userId )
    return getNonValidated(validated)

def getNonValidatedCheckpointsCounts( userId ):
    _nonValidatedCheckpoints = getNonValidatedCheckpoints(userId)
    _counts = []
    for checkpointsList in _nonValidatedCheckpoints:
        _counts.append(len(checkpointsList))
    return _counts

## p(answered question N | answered question P)
<a id=condproba />

In [14]:
# returns all rows of Google form's answers that answered an element of the array 'choice' for question number 'questionIndex'
def getAllAnswerRows(questionIndex, choice):
    return gform[gform.iloc[:, questionIndex].isin(choice)]

def getPercentCorrectPerColumn(_df):
    _count = len(_df)
    _percents = pd.Series(np.full(len(_df.columns), np.nan), index=_df.columns)
    for _rowIndex in _df.index:
        _language = _df['Language'][_rowIndex]
        for _columnName in _df.columns:
            _columnIndex = _df.columns.get_loc(_columnName)
            if ((_columnIndex >= firstEvaluationQuestionIndex) \
                and (_columnIndex < len(_df.columns)-3)):
                if(str(_df[_columnName][_rowIndex]).startswith(str(correctAnswers[_language][_columnIndex]))):
                    if (np.isnan(_percents[_columnName])):
                        _percents[_columnName] = 1;
                    else:
                        _percents[_columnName] = _percents[_columnName]+1
                else:
                    if (np.isnan(_percents[_columnName])):
                        _percents[_columnName] = 0;
                
    _percents = _percents/_count
    _percents['Count'] = _count
    return _percents

def getPercentCorrectKnowingAnswer(questionIndex, choice):
    _answerRows = getAllAnswerRows(questionIndex, choice);
    return getPercentCorrectPerColumn(_answerRows)